### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import json
import music21
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio
import traceback

In [3]:
# parallel
import concurrent
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from fastprogress.fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import MasterBar, ProgressBar

In [4]:
from midi_data import *

In [5]:
from fastai.data_block import get_files

In [6]:
path = Path('data/midi')
orig_path = path/'original'
metapath = path/'metadata'
metapath.mkdir(parents=True, exist_ok=True)

In [7]:
directories = [x for x in path.iterdir() if x.is_dir()]; directories

[PosixPath('data/midi/metadata'),
 PosixPath('data/midi/cprato_midi'),
 PosixPath('data/midi/Wikifonia'),
 PosixPath('data/midi/transposed'),
 PosixPath('data/midi/hymn'),
 PosixPath('data/midi/classic_piano_scrape'),
 PosixPath('data/midi/touhou_scrape'),
 PosixPath('data/midi/hooktheory'),
 PosixPath('data/midi/freemidi_scrape'),
 PosixPath('data/midi/ecomp_scrape'),
 PosixPath('data/midi/midiworld_scrape')]

## General stream parsing

In [8]:
def get_music21_attr(fp, transpose=True, offset=None):
    try:
        stream = music21.converter.parse(fp)
    except Exception as e:
        print('Could not parse stream', fp, e)
#         traceback.print_exc()
        return {}
    stream_attr = get_stream_attr(stream)
    if transpose:
        transposed_file = transpose_midi2c(fp, stream, halfsteps=offset)
        transposed_stream = music21.converter.parse(fp)
        t_key = transposed_stream.flat.analyze('key')
        transposed_attr = {
            'inferred_keyc': f'{t_key.tonic.name} {t_key.mode}',
            'midi_keyc': str(transposed_file),
        }
        stream_attr = {**stream_attr, **transposed_attr}
    return stream_attr 

In [9]:
def get_stream_attr(s):
    instruments = [i.instrumentName for i in list(s.getInstruments(recurse=True)) if i.instrumentName]
    metronome = list(filter(lambda x: isinstance(x, music21.tempo.MetronomeMark), s.flat))[0]
    bpm = metronome.getQuarterBPM()
    s_flat = s.flat
    key = s_flat.analyze('key')
    time_sig = s_flat.timeSignature.ratioString if hasattr(s_flat.timeSignature, 'ratioString') else None
    return {
        'instruments': instruments,
        'bpm': bpm,
        'inferred_key': f'{key.tonic.name} {key.mode}',
        'seconds': s_flat.seconds,
        'time_signature': time_sig,
    }

In [10]:
def process_parallel(func, arr, key_func=None, max_workers=None):
    "Maps function to an array of objects. Key can me modified by passing a key_func"
    results = {}
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(func,o) for i,o in enumerate(arr)]
        for f in progress_bar(concurrent.futures.as_completed(futures), total=len(arr)):
            k,v = f.result()
            results[k] = v
    return results

In [11]:
def parse_songs(data):
    fp = data.get('file_path')
    metadata = data.get('metadata', {})
    offset = data.get('offset', None)
    attr = {}
    try: attr = get_music21_attr(fp, offset=offset)
    except Exception as e: print('Midi Exeption:', fp, e)
    return fp, {**metadata, **attr}

In [12]:
def parse_midi_dir(files, out_path, meta_func, limit=None, recurse=True):
    file2metadata = load_json(out_path)
    if file2metadata is None: file2metadata = {}
        
    if limit: files = files[:limit]
    files = [meta_func(fp) for fp in files if str(fp) not in file2metadata]
    
    parsed = process_parallel(parse_songs, files)
    file2metadata.update(parsed)
    
    json.dump(file2metadata, open(out_path, 'w'))
    
    return file2metadata

In [13]:
import csv
import pandas as pd

def format_values(d):
    def format_value(v):
        if isinstance(v, list): return ','.join(v)
        return v
    return {k:format_value(v) for k,v in d.items()}

def arr2csv(arr, out_file):
    all_keys = {k for d in arr for k in d.keys()}
    arr = [format_values(x) for x in arr]
    with open(out_file, 'w') as f:
        dict_writer = csv.DictWriter(f, list(all_keys))
        dict_writer.writeheader()
        dict_writer.writerows(arr)

### Gather Hooktheory Data

In [16]:
def create_paths(dirname):
    dir_path = orig_path/dirname
    json_path = metapath/f'{dirname}_metadata.json'
    csv_path = metapath/f'{dirname}_metadata.csv'
    return dir_path, json_path, csv_path

In [17]:
ht_path, ht_json, ht_csv = create_paths('hooktheory')
ht_song_list = metapath/'hooktheory_key2info.json'

In [18]:
def song_key(s): return '_'.join(s.parts[-3:-1])

In [19]:
ht_key2info = load_json(ht_song_list)

if ht_key2info is None:
    song_info = list((ht_path/'xml').glob('*/*/*/*.json'))
    ht_key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
    save_json(ht_key2info, ht_song_list)
    len(song_info)

In [20]:
song_json = list((ht_path/'event').glob('*/*/*/*_key.json')); len(song_json) # using json instead of midi for metadata

19876

In [21]:
def get_hooktheory_attr(fp):
    song_info = ht_key2info[song_key(fp)]
    song_json = json.load(open(fp, 'r'))
    metadata = song_json['metadata']
    artist = fp.parts[-3]
    title = fp.parts[-2]
    section = fp.name.split('_')[0]
    midi_path = str(fp.with_suffix('.mid')).replace('event', 'pianoroll').replace('symbol_', '')
    
    # convert stream here
    metadata = {
        'artist': artist,
        'section': section,
        'original_path': midi_path,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'midi_title': metadata['title'],
        'source': 'hooktheory',
        'ht_bpm': metadata['BPM'],
        'ht_mode': metadata['mode'],
        'ht_key': metadata['key'],
        'ht_time_signature': metadata['beats_in_measure']
    }
    mode = metadata['ht_mode']
    if mode is None:
        print('No mode found. Assuming cmajor', fp)
        mode = 'major'
    offset = keyc_offset(metadata['ht_key'], mode)
    return {
        'file_path': midi_path, # midi path not json path
        'metadata': metadata,
        'offset': offset
    }

In [22]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000], key2info); hook_out

In [23]:
ht_metadata = parse_midi_dir(song_json, ht_json, meta_func=get_hooktheory_attr)

No mode found. Assuming cmajor data/midi/hooktheory/event/w/willie-nelson/blue-eyes-cryin/chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/w/weezer/beverly-hills/intro-and-verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/w/woody-guthrie/this-land-is-your-land/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/w/ween/roses-are-free/chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/j/junior-boys/first-time/chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/j/johnny-cash/folsom-prison-blues/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/j/johnny-cash/i-walk-the-line/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/j/jack-johnson/do-you-

No mode found. Assuming cmajor data/midi/hooktheory/event/s/skid-row/wasted-time/outro_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/s/skid-row/wasted-time/pre-outro_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/s/skid-row/wasted-time/pre-chorus-and-chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/s/skid-row/wasted-time/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/s/snow-patrol/run/chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/s/steely-dan/hey-nineteen/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/c/coldplay/viva-la-vida/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/c/cobra-starship/snakes-on-a-plane/chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/c/creedence-clearwater-revival/bad-moon-rising/chorus_symbol_key.json
No mode found. 

No mode found. Assuming cmajor data/midi/hooktheory/event/t/taichi-mukai/reset-%2528kaze-ga-tsuyoku-fuiteiru%2529/intro_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/taichi-mukai/reset/_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/the-waifs/london-still/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/the-four-seasons/sherry/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/the-cure/just-like-heaven/solo_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/tom-hanks/big-soundtrack/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/the-ronettes/be-my-baby/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/the-ronettes/be-my-baby/pre-chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/t/tenacious-d/wonderboy/verse_symbol_key.json
No mode found. Assumin

No mode found. Assuming cmajor data/midi/hooktheory/event/r/r-kelly/i-believe-i-can-fly/chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/r/rihanna/where-have-you-been/intro_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/r/rihanna/love-the-way-you-lie-part-2/verse_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/r/rihanna/love-the-way-you-lie-part-2/pre-chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/r/rihanna/unfaithful/pre-chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/r/rawnald-gregory-erickson-the-second/starfucker/intro_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/r/radiohead/no-surprises/verse-and-pre-chorus_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/event/r/radiohead/where-i-end-and-you-begin/chorus-lead-out_symbol_key.json
No mode found. Assuming cmajor data/midi/hooktheory/eve

Could not parse stream data/midi/hooktheory/pianoroll/w/weezer/beverly-hills/intro-and-verse_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/y/ylvis/the-fox---what-does-the-fox-say/verse-and-pre-chorus_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/j/jay-z/so-ambitious-feat-pharrel-williams/intro_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/j/johnny-cash/folsom-prison-blues/verse_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/j/jack-johnson/do-you-remember/intro-and-verse_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/j/justin-moore/if-heaven-wasnt-so-far-awat/intro-and-verse_key.mid list index out of range
Could not parse stream data/midi/ho

Could not parse stream data/midi/hooktheory/pianoroll/t/tame-impala/pianorollually/verse_key.mid File not found or no such format found for: data/midi/hooktheory/pianoroll/t/tame-impala/pianorollually/verse_key.mid
Could not parse stream data/midi/hooktheory/pianoroll/t/tame-impala/pianorollually/intro_key.mid File not found or no such format found for: data/midi/hooktheory/pianoroll/t/tame-impala/pianorollually/intro_key.mid
Could not parse stream data/midi/hooktheory/pianoroll/t/the-beatles/drive-my-car/verse_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/t/the-beatles/the-end/solo_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/k/katy-pary/fireworks/chorus_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/k/kim-carnes/bette-davis-eyes/chorus_key.mid list index out of range
Could not parse stream data/midi/hooktheory/pianoroll/k/kirbys-adventure---rainbow-resort/nintendo/c

#### Save song_list

In [24]:
ht_metadata = load_json(ht_json)
arr2csv(ht_metadata.values(), ht_csv)
df = pd.read_csv(ht_csv); df.head()

,midi_title,seconds,ht_mode,section,ht_key,midi_keyc,ht_time_signature,ht_bpm,artist,original_path,genres,inferred_keyc,time_signature,source,parts,inferred_key,instruments,song_url,bpm
0,yu-gi-oh3,15.000000,1.0,chorus,C,data/midi/transposed/hooktheory/pianoroll/w/wa...,4,128,wayne-sharpe,data/midi/hooktheory/pianoroll/w/wayne-sharpe/...,NaN,C major,4/4,hooktheory,"intro,chorus",C major,"Piano,Piano",https://www.hooktheory.com/theorytab/view/wayn...,128.0
1,On The Road Again,15.000000,1.0,bridge,E,data/midi/transposed/hooktheory/pianoroll/w/wi...,4,112,willie-nelson,data/midi/hooktheory/pianoroll/w/willie-nelson...,NaN,A major,4/4,hooktheory,"verse,bridge",A major,Piano,https://www.hooktheory.com/theorytab/view/will...,112.0
2,On The Road Again,15.000000,1.0,verse,E,data/midi/transposed/hooktheory/pianoroll/w/wi...,4,112,willie-nelson,data/midi/hooktheory/pianoroll/w/willie-nelson...,NaN,E major,4/4,hooktheory,"verse,bridge",E major,Piano,https://www.hooktheory.com/theorytab/view/will...,112.0
3,Blue Eyes Cryin,28.965517,NaN,chorus,E,data/midi/transposed/hooktheory/pianoroll/w/wi...,4,58,willie-nelson,data/midi/hooktheory/pianoroll/w/willie-nelson...,NaN,E major,4/4,hooktheory,chorus,E major,"Piano,Piano",https://www.hooktheory.com/theorytab/view/will...,58.0
4,Last Christmas Intro,17.777778,1.0,intro,Db,data/midi/transposed/hooktheory/pianoroll/w/wh...,4,108,wham,data/midi/hooktheory/pianoroll/w/wham/last-chr...,Holiday,C# major,4/4,hooktheory,"intro,verse,chorus",C# major,"Piano,Piano",https://www.hooktheory.com/theorytab/view/wham...,108.0


# MIDI directory parsing
hooktheory is a special formatting case

### Gather FreeMidi

In [25]:
fm_path = path/'freemidi_scrape'
fm_dance_path = metapath/f'freemidi_dance_metadata.json'
fm_pop_path = metapath/f'freemidi_pop_metadata.json'
fm_csv_path = metapath/'freemidi_metadata.csv'
list(fm_path.glob('*'))

[PosixPath('data/midi/freemidi_scrape/genre-disco'),
 PosixPath('data/midi/freemidi_scrape/genre-pop'),
 PosixPath('data/midi/freemidi_scrape/genre-dance-eletric'),
 PosixPath('data/midi/freemidi_scrape/genre-punk'),
 PosixPath('data/midi/freemidi_scrape/genre-hip-hop-rap'),
 PosixPath('data/midi/freemidi_scrape/genre-rock')]

In [30]:
def parse_freemidi_songs(fp, genre=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1]
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genre': genre,
        'source': source
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [31]:
# sanity check
# dance_files = (freemidi_path/'genre-dance-eletric/').ls()
# freemidi_out = parse_freemidi_songs(dance_files[0], genre='dance'); freemidi_out

In [32]:
d_parse_func = partial(parse_freemidi_songs, genre='dance', source='freemidi')
dir_path = fm_path/'genre-dance-eletric'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_dance_list = parse_midi_dir(file_list, fm_dance_path, meta_func=d_parse_func)

Midi Exeption: data/midi/freemidi_scrape/genre-dance-eletric/Tori Amos - Not The Red Baron.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - Freelove.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/freemidi_scrape/genre-dance-eletric/Bjork - Glora.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - I'm Not Givin' You Up.mid badly formated midi bytes, got: b'RIFFB\x8c\x00\x00RMIDdata~\x8b\x00\x00'
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Atomic Kitten - Whole Again.mid badly formated midi bytes, got: b'RIFF\x08K\x00\x00RMIDdata{J\x00\x00'
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Apollo 440 - Los

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=8, channel=None>: unknown midi event type: None; ignored.


Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Tatu - Ya Tvoy Vrag (I'm Your Enemy).mid badly formated midi bytes, got: b'RIFF,\xa3\x00\x00RMIDdata\xc6\xa2\x00\x00'
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Get On Your Feet.mid badly formated midi bytes, got: b'RIFF\x92\xa8\x01\x00RMIDdata\xd8\xa7\x01\x00'
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Mi Tierra.mid badly formated midi bytes, got: b'RIFFz\r\x01\x00RMIDdata\xce\x0c\x01\x00'
Midi Exeption: data/midi/freemidi_scrape/genre-dance-eletric/Tori Amos - 10000 Oceans.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - Breathe.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/freemidi_scrape/genre-dance-eletric/Radiohead - How 

Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Akcent - Dajcie Mi Gitarę.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Akcent - Dajcie Mi Gitarę.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Robbie Williams - Morning Sun.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Robbie Williams - Morning Sun.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Radiohead - Big Ideas.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Radiohead - Big Ideas.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Gwen Stefani - The Sweet Escape.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Gwen Stefani - The Sweet Escape.mid
Midi Exeption: data/midi/freemidi_scrape/genre-dance-eletric/DJ Bobo - Dream.mid [Errno 2] No such file or directory: 'data/midi/freemidi_scrape/genre-dance-eletric/DJ Bobo - Dream.mid'
Could not parse stream

Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Yellow Magic Orchestra - Mass.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Yellow Magic Orchestra - Mass.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Madonna - Justify My Love.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Madonna - Justify My Love.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Radiohead - A Punchup At A Wedding.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Radiohead - A Punchup At A Wedding.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - Christmas Island.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - Christmas Island.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Madonna - Hung Up.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Madonna - Hung Up.mid
Mid

...

Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Tatu - Malchik gey.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Tatu - Malchik gey.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Culture Beat - Anything.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Culture Beat - Anything.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Robbie Williams - She's The One.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Robbie Williams - She's The One.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Yellow Magic Orchestra - Cosmic Surfin.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Yellow Magic Orchestra - Cosmic Surfin.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/New Order - Thieves Like Us.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/New Order - Thieves Like Us.mid
C

Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Take That - Back For Good.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Take That - Back For Good.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Daft Punk - The Son Of Flyn.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Daft Punk - The Son Of Flyn.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Our Daughters Wedding - Love Machine.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Our Daughters Wedding - Love Machine.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Tori Amos - Baker Baker.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Tori Amos - Baker Baker.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Whitney Houston - Didn't We Almost Have It All.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Whitney Hous

Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Robbie Williams - By all means necessary.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Robbie Williams - By all means necessary.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Chemical Brothers - Let Forever Be.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Chemical Brothers - Let Forever Be.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/20 Fingers - Short Dick Man.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/20 Fingers - Short Dick Man.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Alanis Morissette - Unsent.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Alanis Morissette - Unsent.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Robbie Williams - Come Undone.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletr

Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Alanis Morissette - Mary Jane.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Alanis Morissette - Mary Jane.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Captain Jack - Drill Instructor.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Captain Jack - Drill Instructor.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Owl City - Honey And The Bee.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Owl City - Honey And The Bee.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Radiohead - Creep Acoustic.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Radiohead - Creep Acoustic.mid
Could not parse stream data/midi/freemidi_scrape/genre-dance-eletric/Avicii - The Days.mid Cannot find file in data/midi/freemidi_scrape/genre-dance-eletric/Avicii - The Days.mid
Could not par

KeyboardInterrupt: 

In [62]:
p_parse_func = partial(parse_freemidi_songs, genre='pop', source='freemidi')
dir_path = fm_path/'genre-pop'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_pop_list = parse_midi_dir(file_list, fm_pop_path, meta_func=p_parse_func)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/fastprogress/fastprogress.py:95: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [66]:
fmd = load_json(fm_dance_path)
fmp = load_json(fm_pop_path)
fm_all = list(fmd.values())+list(fmp.values())
arr2csv(fm_all, fm_csv_path)
df = pd.read_csv(fm_csv_path); df.head()

,seconds,artist,inferred_keyc,midi,time_signature,midi_keyc,bpm,inferred_key,instruments,title,genre,source
0,28.732394,Tune Up,F major,../data/midi/freemidi_scrape/genre-dance-eletr...,NaN,../data/midi/transposed/freemidi_scrape/genre-...,142.0,F major,NaN,Bounce,dance,freemidi
1,138.461538,Radiohead,C major,../data/midi/freemidi_scrape/genre-dance-eletr...,4/4,../data/midi/transposed/freemidi_scrape/genre-...,78.0,C major,"Piano,Electric Bass",Banana Co,dance,freemidi
2,150.428571,Madonna,F minor,../data/midi/freemidi_scrape/genre-dance-eletr...,4/4,../data/midi/transposed/freemidi_scrape/genre-...,160.0,F minor,"Recorder,Vibraphone",Dress You Up,dance,freemidi
3,122.888889,Aqua,E- major,../data/midi/freemidi_scrape/genre-dance-eletr...,4/4,../data/midi/transposed/freemidi_scrape/genre-...,135.0,E- major,"Guitar,Electric Bass,Electric Guitar,StringIns...",Dr Jones,dance,freemidi
4,256.800000,Garbage,G minor,../data/midi/freemidi_scrape/genre-dance-eletr...,4/4,../data/midi/transposed/freemidi_scrape/genre-...,100.0,G minor,Clarinet,Queer,dance,freemidi


### Gather Cprato

In [ ]:
ht_path, ht_json, ht_csv = create_paths('cprato_midi')
ht_song_list = metapath/'hooktheory_key2info.json'

In [86]:
cp_path = path/'cprato_midi'
cp_json_path = cp_path/f'cprato_metadata.json'
cp_csv_path = 'cprato_metadata.csv'
list(cp_path.glob('*'))[:5]

[PosixPath('../data/midi/cprato_midi/Basto - Again And Again (midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../data/midi/cprato_midi/The Weeknd ft. Lana Del Rey - Stargirl Interlude  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../data/midi/cprato_midi/Two Steps From Hell - Magic of Love  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../data/midi/cprato_midi/Bermuda Loverz - My Girl (Ladidada) (Rimini Rockaz Radio Edit) (Midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../data/midi/cprato_midi/Cascada - Everytime We Touch (Midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../data/midi/cprato_midi/Tujamo - Drop That Low  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../data/midi/cprato_midi/ODESZA feat. Madelyn Grant - Sun Models  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../data/midi/cprato_midi/John Powell - Test Drive (How To Train Your Dragon)  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('../

In [78]:
def parse_cprato_songs(fp, genre=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1].replace('(midi by Carlo Prato) (www.cprato.com)', '')
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genre': genre,
        'source': source
    }

In [81]:
cp_meta = partial(parse_cprato_songs, genre='EDM,inferred', source='cprato')
file_list = get_files(cp_path, extensions=['.mid'], recurse=True)
cp_md = parse_midi_dir(file_list, cp_json_path, cp_meta)

#### CSV

In [87]:
cp = load_json(cp_json_path)
arr2csv(cp.values(), cp_csv_path)
df = pd.read_csv(cp_csv_path); df.head()

,seconds,artist,inferred_keyc,midi,time_signature,midi_keyc,bpm,inferred_key,instruments,title,genre,source
0,36.756757,John Powell,D major,../data/midi/cprato_midi/John Powell - Test Dr...,4/4,../data/midi/transposed/cprato_midi/John Powel...,111.0,D major,"StringInstrument,StringInstrument,StringInstru...",Test Drive (How To Train Your Dragon),"EDM,inferred",cprato
1,29.142857,Basshunter,F major,../data/midi/cprato_midi/Basshunter - So Near ...,NaN,../data/midi/transposed/cprato_midi/Basshunter...,140.0,F major,NaN,So Near So Close (Midi By Carlo Prato) (www.cp...,"EDM,inferred",cprato
2,30.857143,Drunken Monkey,G minor,../data/midi/cprato_midi/Drunken Monkey - E (M...,NaN,../data/midi/transposed/cprato_midi/Drunken Mo...,140.0,G minor,NaN,E (Midi By Carlo Prato) (www.cprato.com),"EDM,inferred",cprato
3,51.147541,Nora En Pure,F minor,../data/midi/cprato_midi/Nora En Pure - Mornin...,4/4,../data/midi/transposed/cprato_midi/Nora En Pu...,122.0,F minor,"Piano,Piano,Piano,Piano,Piano,Piano",Morning Dew (Original Mix),"EDM,inferred",cprato
4,28.531469,Zane Foster,B major,../data/midi/cprato_midi/Zane Foster - Big Boo...,NaN,../data/midi/transposed/cprato_midi/Zane Foste...,143.0,B major,NaN,Big Boom Bang (Rob Mayth Remix),"EDM,inferred",cprato


### Gather MidiWorld

In [89]:
mw_path = path/'midiworld_scrape'
mw_json_path = mw_path/f'midiworld_metadata.json'
mw_csv_path = 'midiworld_metadata.csv'
list(mw_path.glob('*'))[:5]

[PosixPath('../data/midi/midiworld_scrape/named_midi'),
 PosixPath('../data/midi/midiworld_scrape/unknown_midi')]

In [91]:
def parse_midiworld_songs(fp, genre=None, source=None):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genre': genre,
        'source': source
    }

In [94]:
mw_meta = partial(parse_midiworld_songs, source='midiworld')
file_list = get_files(mw_path/'named_midi', extensions=['.mid'], recurse=True)
mw_md = parse_midi_dir(file_list, mw_json_path, mw_meta)

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-5, velocity=100>: Problem with representing either -5 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=100>: Problem with representing either -1 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-3, velocity=100>: Problem with representing either -3 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-1, velocity=80>: Problem with representing either -1 or 80; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=100>: Problem with representing either -1 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-3, velocity=80>: Problem with representing either -3 or 80; ignored.
midi: WARNING: Conversion erro

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

...

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=3, channel=16, pitch=-1, velocity=1>: Problem with representing either -1 or 1; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=3, channel=16, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=3, channel=1, pitch=-1, velocity=1>: Problem with representing either -1 or 1; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=3, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=4, channel=16, pitch=-1, velocity=1>: Problem with representing either -1 or 1; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=4, channel=16, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent N

Midi Exeption: ../data/midi/midiworld_scrape/named_midi/National_Anthems_-_Denmark.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.


midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=16, pitch=-3, velocity=1>: Problem with representing either -3 or 1; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=16, pitch=-3, velocity=0>: Problem with representing either -3 or 0; ignored.


Could not parse stream ../data/midi/midiworld_scrape/named_midi/Nirvana_-_Milk_It.mid badly formed midi string: missing leading MTrk
Midi Exeption: ../data/midi/midiworld_scrape/named_midi/National_Anthems_-_Japan.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.


midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=9, channel=11, pitch=-4, velocity=64>: Problem with representing either -4 or 64; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=9, channel=11, pitch=-4, velocity=0>: Problem with representing either -4 or 0; ignored.


In [ ]:
mw = load_json(mw_json_path)
arr2csv(mw.values(), mw_csv_path)
df = pd.read_csv(mw_csv_path); df.head()

### Yamaha - piano

In [95]:
ec_path = path/'ecomp_scrape'
ec_song = ec_path/'song_list.json'
ec_json_path = mw_path/f'ecomp_metadata.json'
ec_csv_path = 'ecomp_metadata.csv'
list(ec_path.glob('*'))[:5]

In [91]:
def parse_midiworld_songs(fp, genre=None, source=None):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genre': genre,
        'source': source
    }

In [94]:
mw_meta = partial(parse_midiworld_songs, source='midiworld')
mw_md = parse_midi_dir(mw_path/'named_midi', mw_json_path, mw_meta)

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-5, velocity=100>: Problem with representing either -5 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=100>: Problem with representing either -1 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-3, velocity=100>: Problem with representing either -3 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-1, velocity=80>: Problem with representing either -1 or 80; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=100>: Problem with representing either -1 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-3, velocity=80>: Problem with representing either -3 or 80; ignored.
midi: WARNING: Conversion erro

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=0>: Problem with representing either -1 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_

...

Midi Exeption: ../data/midi/midiworld_scrape/named_midi/National_Anthems_-_Papua_New_Guinea.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: ../data/midi/midiworld_scrape/named_midi/Veruca_Salt_-_Volcana_Girls.mid list index out of range


midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=14, channel=12, pitch=-2, velocity=1>: Problem with representing either -2 or 1; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=14, channel=12, pitch=-2, velocity=0>: Problem with representing either -2 or 0; ignored.


Midi Exeption: ../data/midi/midiworld_scrape/named_midi/National_Anthems_-_Australia.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Could not parse stream ../data/midi/midiworld_scrape/named_midi/Bomfunk_MCs_-_B-Boys_And_Fly-Girls.mid badly formated midi bytes, got: b'RIFF^\xd8\x00\x00RMIDdata\xa1\xd7\x00\x00'
Could not parse stream ../data/midi/midiworld_scrape/named_midi/Nightwish_-_Feel_For_You.mid badly formated midi bytes, got: b'RIFFr\x9a\x00\x00RMIDdatae\x9a\x00\x00'


KeyboardInterrupt: 

Midi Exeption: ../data/midi/midiworld_scrape/named_midi/98_Degrees_-_The_Hardest_Thing.mid list index out of range
Midi Exeption: ../data/midi/midiworld_scrape/named_midi/Leo_Sayer_-_You_make_me_feel_like_Dancn.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: ../data/midi/midiworld_scrape/named_midi/Firefall_-_Just_Remember_I_Love_you.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.


In [ ]:
mw = load_json(mw_json_path)
arr2csv(mw.values(), mw_csv_path)
df = pd.read_csv(mw_csv_path); df.head()

### Classic Piano

In [ ]:
classic_scrape_path = Path('../data/midi/classic_piano_scrape')

### Creating CSV

In [ ]:
import csv
toCSV = [{'name':'bob','age':25,'weight':200},
         {'name':'jim','age':31,'weight':180}]
keys = toCSV[0].keys()
with open('people.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)